<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using SingleStore with OpenAI's Swarm</h1>
    </div>
</div>

In [7]:
!pip cache purge --quiet

In [8]:
!pip install git+https://github.com/openai/swarm.git --quiet
!pip install langchain --quiet
!pip install langchain-community --quiet
!pip install langchain-openai --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six --quiet
!pip install unstructured==0.10.14 --quiet

In [9]:
import nltk
import shutil

from langchain.document_loaders import OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import SingleStoreDB
from langchain_openai import OpenAIEmbeddings
from singlestoredb.management import get_secret
from swarm import Swarm, Agent

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [10]:
from sqlalchemy import *

conn = create_engine(connection_url)

In [11]:
nltk.download("punkt_tab")
nltk.download("averaged_perceptron_tagger_eng")

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [12]:
loader = OnlinePDFLoader("https://www.investni.com/sites/default/files/2021-02/NI-fintech-document.pdf")

data = loader.load()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [13]:
print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

You have 1 document(s) in your data
There are 39674 characters in your document


In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 20
)
texts = text_splitter.split_documents(data)

print (f"You have {len(texts)} pages")

You have 23 pages


In [15]:
os.environ["OPENAI_API_KEY"] = get_secret("OPENAI_API_KEY")

In [16]:
%%sql
DROP TABLE IF EXISTS fintech_docs;

++
||
++
++

In [17]:
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

docsearch = SingleStoreDB.from_documents(
    texts,
    embedding,
    table_name = "fintech_docs"
)

In [18]:
# Define the search function
def search_s2(query_text: str) -> str:
    # Perform a similarity search and return the first result's content
    docs = docsearch.similarity_search(query_text)
    return docs[0].page_content

# Define a function to transfer control to Agent B
def transfer_to_agent_b():
    return agent_b

In [19]:
# Define shared context to hold information between agents
shared_context = {}

# Define Agent A to focus on blockchain context
agent_a = Agent(
    name = "Blockchain Expert",
    model = "gpt-4o-mini",
    instructions = "You are an expert in blockchain technology.",
    functions = [search_s2, transfer_to_agent_b]
)

# Define Agent B to focus on investment opportunities
agent_b = Agent(
    name = "Investment Expert",
    model = "gpt-4o-mini",
    instructions = "You are an expert in stock market investment."
)

In [20]:
client = Swarm()

# Run Agent A with a blockchain-specific query
response_a = client.run(
    agent = agent_a,
    messages = [{
        "role": "user",
        "content": "What are the latest developments and trends in blockchain technology in Northern Ireland?"
    }],
)

# Store Agent A's response in shared context for Agent B to expand on
shared_context["blockchain_context"] = response_a.messages[-1]["content"]

# Output the response from Agent A
print(shared_context["blockchain_context"])

Northern Ireland has recently established itself as a prominent center for blockchain technology and FinTech innovations, driven by several key developments and trends:

1. **Growth of FinTech Sector**: Northern Ireland has become a leading global location for FinTech investments. The financial services sector is supported by world-class talent and competitive operating costs, which appeal to international firms.

2. **Investment from Global Firms**: Major companies, including Citi, Allstate, Liberty Mutual, and FD Technologies, are expanding their operations in Northern Ireland. Recent entrants like CME Group, Vela, and TP ICAP have also set up significant delivery centers, further boosting the local tech ecosystem.

3. **Thriving Start-up Ecosystem**: The entrepreneurial scene is flourishing, supported by a vibrant network that promotes collaboration among tech start-ups and SMEs. This environment is conducive to innovation and the adoption of technologies like blockchain.

4. **Tech

In [22]:
# Run Agent B with a focus on investment opportunities within the blockchain context
response_b = client.run(
    agent = agent_b,
    messages = [{
        "role": "user",
        "content": f"{shared_context['blockchain_context']} Based on these developments, what are the best investment opportunities in blockchain in Northern Ireland?"
    }],
)

# Output the final response from Agent B
print(response_b.messages[-1]["content"])

Northern Ireland's emergence as a hub for blockchain technology and FinTech innovations presents several investment opportunities. Given the outlined trends and developments, here are some potential avenues for investment:

1. **Blockchain Start-ups**: Investing in local blockchain start-ups that are focused on developing unique applications, platforms, or services can yield significant returns as these companies scale. Look for ventures that focus on areas such as decentralized finance (DeFi), supply chain transparency, digital identity solutions, or voter/surveillance technologies.

2. **Strategic Partnerships with Global Firms**: Given the increased presence of global firms in the region, investment opportunities may arise from partnerships or collaborations with these companies. Consider investing in existing local firms that are forming partnerships with larger entities in areas like compliance technologies, payments processing, or smart contracts.

3. **Innovation Hubs and Incuba

## Cleanup

In [23]:
shutil.rmtree("nltk_data")